<a href="https://colab.research.google.com/github/HaeunJeon/Jeju_Rec/blob/main/%EC%A0%9C%EC%A3%BC_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import pandas as pd

In [ ]:
test_data=pd.read_csv("/content/drive/MyDrive/종합설계/labeled_jeju")

In [ ]:
train_data.columns=['text','name','label']
train_data.head()

,text,name,label
0,청정 우도에서 만난 청정 노을의 맑은 색,우도(해양도립공원),2
1,보리밭을 지나다 아름다운 풍경에 홀릭 마침 남편의 생일을 맞아 멋진 사진을 남겨 ...,우도(해양도립공원),2
2,보리밭을 지나다 아름다운 풍경에 홀릭 마침 남편의 생일을 맞아 멋진 사진을 남겨 ...,우도(해양도립공원),2
3,우도섬에 봄이 왔습니다 유채와 청보리 파란 하늘과 그보다 더 푸른섬 우도가 제주의 ...,우도(해양도립공원),2
4,일몰이 제일 아름답다는 우도 낮에 사람들로 가득한 우도는 마지막 배가 떠나고 나면 ...,우도(해양도립공원),2


In [ ]:
train_data['text'] = train_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,text,name,label
0,청정 우도에서 만난 청정 노을의 맑은 색,우도(해양도립공원),2
1,보리밭을 지나다 아름다운 풍경에 홀릭 마침 남편의 생일을 맞아 멋진 사진을 남겨 ...,우도(해양도립공원),2
2,보리밭을 지나다 아름다운 풍경에 홀릭 마침 남편의 생일을 맞아 멋진 사진을 남겨 ...,우도(해양도립공원),2
3,우도섬에 봄이 왔습니다 유채와 청보리 파란 하늘과 그보다 더 푸른섬 우도가 제주의 ...,우도(해양도립공원),2
4,일몰이 제일 아름답다는 우도 낮에 사람들로 가득한 우도는 마지막 배가 떠나고 나면 ...,우도(해양도립공원),2


In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [ ]:
X_train = []
for sentence in train_data['text']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 9050
등장 빈도가 2번 이하인 희귀 단어의 수: 5203
단어 집합에서 희귀 단어의 비율: 57.491712707182316
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 4.953017874632713


In [ ]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 3848


In [ ]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
print(X_train[:3])

[[1460, 57, 8, 811, 1460, 384, 313, 775], [2022, 1, 776, 20, 70, 2779, 1545, 850, 1194, 1, 371, 68, 23, 1, 991, 181, 213, 21, 1, 3219, 23, 55, 670, 3220, 64, 102, 5, 20, 1460, 1634, 8, 1861, 20, 1461, 272, 62], [2022, 1, 776, 20, 70, 2779, 1545, 850, 1194, 1, 371, 68, 23, 1, 991, 181, 213, 21, 1, 3219, 23, 55, 670, 3220, 64, 102, 5, 20, 1460, 1634, 8, 1861, 20, 1461, 272, 62]]


# **인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(train_data['label']) #인코더에 맞게 변환중 
train_data['label']=y # 하고 추가 

In [ ]:
y_train = np.array(train_data['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

5918
5918


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
#최대 길이 지정해 주려고 
max_len = 100
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 100 이하인 샘플의 비율: 99.25650557620817


In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))# Add a LSTM layer with 128 internal units.
model.add(Dense(3, activation='softmax'))#Dense 레이어의 유닛수가 클래스 수에 맞게 수정해야함

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) #가장 성능이 좋은 모델을 가져오는 과정

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
79/79 [==============================] - ETA: 0s - loss: 0.7403 - acc: 0.6935
Epoch 1: val_acc improved from -inf to 0.01182, saving model to best_model.h5
79/79 [==============================] - 44s 516ms/step - loss: 0.7403 - acc: 0.6935 - val_loss: 1.3823 - val_acc: 0.0118
Epoch 2/15
79/79 [==============================] - ETA: 0s - loss: 0.3873 - acc: 0.8576
Epoch 2: val_acc improved from 0.01182 to 0.49240, saving model to best_model.h5
79/79 [==============================] - 23s 293ms/step - loss: 0.3873 - acc: 0.8576 - val_loss: 1.2857 - val_acc: 0.4924
Epoch 3/15
79/79 [==============================] - ETA: 0s - loss: 0.2433 - acc: 0.9252
Epoch 3: val_acc improved from 0.49240 to 0.57348, saving model to best_model.h5
79/79 [==============================] - 23s 295ms/step - loss: 0.2433 - acc: 0.9252 - val_loss: 1.1167 - val_acc: 0.5735
Epoch 4/15
79/79 [==============================] - ETA: 0s - loss: 0.1643 - acc: 0.9470
Epoch 4: val_acc improved from 0.57348

# **테스트 데이터 가져오기**

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/종합설계/테스트 원본.xlsx')
df.to_csv('test')
test_data=pd.read_csv("/content/test")

In [ ]:
test_data = test_data.loc[:,['place', 'label', 'text']]
test_data.columns=['name','label','text']
test_data.tail()

,name,label,text
40,아쿠아플라넷,레저/체험,엄청난 규모로 대형 수족관이 있고 다양한 물고기를 볼 수 있다. 환상적인 공연도 있...
41,수월봉 지질트레일,레저/체험,"경관이 아름다우며, 유네스코에서 지정한 세계지질공원이다. 수려한 지질자원과 풍경을 ..."
42,오설록티뮤지엄,레저/체험,푸른 녹차밭이 펼쳐지는 제주도 서광다원 입구에 위치하고 있다. 동서양 전통과 현대가...
43,제주 유리의 성,레저/체험,"유리로 만든 많은 조형물, 공예품, 작품들을 볼 수 있다. 예쁘게 꾸며져 있고 직접..."
44,제주 더 마 파크,레저/체험,"카트, 승마, 공연, 동물원이 모여있다. 스피드 있는 카트를 탈 수 있고 다양한 묘..."


In [ ]:
test_data['text'] = test_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data['text'] = test_data['text'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_data['text'].replace('', np.nan, inplace=True)
print(test_data.isnull().sum())

name     0
label    0
text     0
dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
X_test = []
for sentence in test_data['text']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
y_test = np.array(test_data['label'])

In [ ]:
y_test

array(['자연', '자연', '자연', '자연', '자연', '자연', '자연', '자연', '자연', '자연', '자연',
       '자연', '자연', '자연', '자연', '교육', '교육', '교육', '교육', '교육', '교육', '교육',
       '교육', '교육', '교육', '교육', '교육', '교육', '교육', '교육', '레저/체험', '레저/체험',
       '레저/체험', '레저/체험', '레저/체험', '레저/체험', '레저/체험', '레저/체험', '레저/체험',
       '레저/체험', '레저/체험', '레저/체험', '레저/체험', '레저/체험', '레저/체험'], dtype=object)

In [ ]:
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

In [ ]:
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
loaded_model = load_model('best_model.h5')

**바꾼 부분 >> encoder 에 y_test 핏 트랜스폼 시킴**

In [ ]:
y_1=encoder.fit_transform(test_data['label']) #인코더에 맞게 변환중
y_test=y_1 

In [ ]:
y_test

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

2/2 [==============================] - 1s 31ms/step - loss: 0.3540 - acc: 0.8889

 테스트 정확도: 0.8889


In [ ]:
softmax = loaded_model.predict(X_test)

In [ ]:
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.6f}".format(x)})

In [ ]:
softmax = list(softmax)

In [ ]:
les = []
edu = []
nature = []

for i in range(45):
  for j in range(3):
    if j == 0: edu.append(softmax[i][j])
    elif j ==1 : les.append(softmax[i][j])
    elif j == 2 : nature.append(softmax[i][j])
      

In [ ]:
test_data['edu'] = edu
test_data['les'] = edu
test_data['nature'] = nature

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
test_data = test_data.loc[:,['name', 'edu', 'les','nature']]
test_data

,name,edu,les,nature
0,성산 일출봉,0.00000,0.00000,0.99999
1,사려니숲길,0.00001,0.00001,0.99993
2,협재 해수욕장,0.00002,0.00002,0.99987
3,월정리 해변,0.00012,0.00012,0.99932
4,함덕 해수욕장,0.00021,0.00021,0.99796
5,산굼부리,0.00005,0.00005,0.99966
6,섭지코지,0.00002,0.00002,0.99988
7,한라산국립공원,0.16784,0.16784,0.79822
8,비자림,0.00020,0.00020,0.99953
9,천지연폭포,0.00001,0.00001,0.99993


In [ ]:
test_data.to_csv("/content/drive/MyDrive/종합설계/softmax", encoding='utf-8-sig', index=False)